# 3rd model to train image classifier
> using Keras and Tensorflow

**Import the libraries and prepare the dataset.**

In [317]:
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import random
# from keras.layers import Conv2D, MaxPooling2D
# from keras.layers import Activation, Dropout, Flatten, Dense

print(tf.__version__)
image_size = 100

1.11.0


In [318]:
import os 
from tensorflow.keras.preprocessing import image
 
import pathlib
def getImagePaths(rootPath):
  data_root = pathlib.Path(rootPath)
  image_paths = list(data_root.glob('*/*'))
  image_paths = [str(path) for path in image_paths if not str(path).endswith('DS_Store')]
  random.shuffle(image_paths)
  return image_paths

#image resize -> PIL -> tensorflow 
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=1)
  image = tf.image.resize_images(image, [image_size, image_size])
  image /= 255.0  # normalize to [0,1] range
  return image

def load_and_preprocess_image(path):
  image = tf.read_file(path)
  return preprocess_image(image)


In [319]:
# Prepare the train Dataset
all_image_paths = getImagePaths('./data/train/')
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
print(label_names)
label_to_index = dict((name, index) for index,name in enumerate(label_names))
print(label_to_index)

all_image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_image_paths]
number_of_images = len(all_image_labels)
all_image_array = []
for image_path in all_image_paths:
  all_image_array.append(load_and_preprocess_image(image_path))

all_image_tensor = tf.convert_to_tensor(all_image_array, dtype=tf.float32)
train_dataset_reshape = tf.reshape(all_image_tensor, [number_of_images, image_size, image_size, 1])
train_dataset_reshape.shape

# Prepare the prediction Dataset
all_predict_paths = getImagePaths('./data/test/')
all_predict_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_predict_paths]
number_of_predict = len(all_predict_labels)
all_predict_array = []
for predict_path in all_predict_paths:
  all_predict_array.append(load_and_preprocess_image(predict_path))

all_predict_tensor = tf.convert_to_tensor(all_predict_array, dtype=tf.float32)
predict_dataset_reshape = tf.reshape(all_predict_tensor, [number_of_predict, image_size, image_size, 1])
predict_dataset_reshape.shape
print(number_of_predict)

['barramundi', 'bream', 'snapper']
{'barramundi': 0, 'snapper': 2, 'bream': 1}
4


In [320]:
# image_path = all_image_paths[1]
# print(image_path)
# x = load_and_preprocess_image(image_path)
# x.shape
# plt.imshow(load_and_preprocess_image(image_path))
# plt.grid(False)
# plt.show()

**Setup the model and train it**

In [321]:
# model = keras.Sequential([
#     keras.layers.Flatten(input_shape=(image_size, image_size, 1)),
#     keras.layers.Dense(64, activation=tf.nn.relu),
#     keras.layers.Dense(3, activation=tf.nn.softmax)
# ])

# layer1
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), input_shape=(image_size, image_size, 1)))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
#layer2
model.add(keras.layers.Conv2D(64, (3, 3)))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(3, 3)))
#layer3
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dropout(0.5))
#layer4
model.add(keras.layers.Dense(3))
model.add(keras.layers.Activation('sigmoid'))

In [322]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [323]:
model.fit(train_dataset_reshape, all_image_labels, epochs=50, steps_per_epoch=1)

Epoch 1/50
1/1 [==============================] - 3s 3s/step - loss: 1.1036 - acc: 0.3333
Epoch 2/50
1/1 [==============================] - 0s 367ms/step - loss: 1.2138 - acc: 0.3333
Epoch 3/50
1/1 [==============================] - 0s 355ms/step - loss: 1.1458 - acc: 0.3333
Epoch 4/50
1/1 [==============================] - 0s 355ms/step - loss: 1.0859 - acc: 0.3889
Epoch 5/50
1/1 [==============================] - 0s 347ms/step - loss: 1.0814 - acc: 0.4722
Epoch 6/50
1/1 [==============================] - 0s 355ms/step - loss: 1.0637 - acc: 0.4722
Epoch 7/50
1/1 [==============================] - 0s 359ms/step - loss: 1.1001 - acc: 0.3056
Epoch 8/50
1/1 [==============================] - 0s 359ms/step - loss: 1.0643 - acc: 0.4722
Epoch 9/50
1/1 [==============================] - 0s 362ms/step - loss: 1.0715 - acc: 0.3889
Epoch 10/50
1/1 [==============================] - 0s 339ms/step - loss: 1.0842 - acc: 0.4167
Epoch 11/50
1/1 [==============================] - 0s 350ms/step - loss:

Test the model and make use of that.

In [324]:
test_loss, test_acc = model.evaluate(train_dataset_reshape, all_image_labels, steps=1)
print('Test accuracy:', test_acc)

1/1 [==============================] - 1s 1s/step
Test accuracy: 1.0


In [325]:
predictions = model.predict(predict_dataset_reshape, steps=1)
for item in predictions:
  print(np.argmax(item), end=', ')
print()
print(all_predict_labels)

2, 1, 1, 2, 
[0, 1, 1, 2]
